#WELCOME
In this Assignment, you'll use **Classifier** to predict the output. You are allowed to write/edit between the
```# START CODE HERE``` & ```# END CODE HERE``` only.<br>
Don't edit the test cells otherwise you'd be failed in the assignment.

Upload the ```utils.zip``` file before running the 1st cell.
[Demo](https://drive.google.com/file/d/1ebkQR6ikjGottmxa5KL83ZWdjTLvRxbm/view?usp=share_link)

**BEST OF LUCK**

**RUN THIS CODE TO UNIZP THE REQUIRED FILES**

In [1]:
!unzip -j /content/utils.zip -d.

unzip:  cannot find or open /content/utils.zip, /content/utils.zip.zip or /content/utils.zip.ZIP.


# Dataset Loading and Pre-Processing

In [2]:
import pandas as pd
data = pd.read_csv("/content/weather_classification_data.csv")

## Data Info
```data.info``` gives info about data-types and number of null objects.  
There are no null data, no need to remove any.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


## Data Shuffling
Data Shuffling removes any bias present and creates uniform distribution.

In [4]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
6808,40.0,23,4.0,4.0,partly cloudy,1013.37,8,Summer,8.0,mountain,Sunny
13000,-6.0,65,6.0,85.0,overcast,980.58,0,Winter,2.0,mountain,Snowy
1699,-5.0,79,2.0,61.0,overcast,982.37,1,Winter,5.0,mountain,Snowy
10670,23.0,56,5.5,15.0,partly cloudy,1009.52,4,Winter,7.0,mountain,Cloudy
5576,32.0,23,5.5,73.0,cloudy,1137.23,8,Winter,6.0,inland,Sunny


## Scaling and Labeling
Machine can't understand string so all the object datatypes need to be LABELED with integers.<br><br>
Why SCALING?<br>
You might have noticed one column contains data with large values (of order 1000) whereas some column contain small value data (of order 10). As a result, large value columns get more power/importance than others. To remove this disparity we scaled them to same order/values.

**CODE**

We need to find the columns with object datatypes.

HINT: Use ```data.select_dtypes().columns.tolist()```<br>Reference: [w3schools](https://www.w3schools.com/python/pandas/ref_df_select_dtypes.asp#:~:text=The%20select_dtypes()%20method%20returns,the%20specified%20dtype(s).&text=Note%3A%20You%20must%20specify%20at,you%20will%20get%20an%20error.)

In [5]:
# START CODE HERE
non_object_columns = data.select_dtypes(include = ['float64','int64']).columns.tolist()
object_columns = data.select_dtypes(object).columns.tolist()
# END CODE HERE
print(f"Object Columns: {object_columns}\nNon Object Columns: {non_object_columns}")

Object Columns: ['Cloud Cover', 'Season', 'Location', 'Weather Type']
Non Object Columns: ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']


**TEST** (DON'T EDIT THIS CELL)

In [6]:
# DON'T EDIT THIS CELL
if (object_columns == ['Cloud Cover', 'Season', 'Location', 'Weather Type'] and non_object_columns == ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


**CODE**

We need to convert the object-type columns data into integral labels and scale the non-object-type columns data.

HINT: Use ```ColumnTransformer()```<br>Reference: [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), [ChatGPT](https://chatgpt.com/c/7f3c1b02-5da0-4c34-bdb2-d999dccb9c2f)

In [14]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# START CODE HERE

transformers = [
    ('ordinal', OrdinalEncoder(), object_columns),  # OrdinalEncoder for object-type columns
    ('scaler', StandardScaler(), non_object_columns)  # StandardScaler for non-object-type columns
]

# Create the ColumnTransformer
column_transformer = ColumnTransformer(transformers)

# Fit and transform the data
data_scaled_labeled = column_transformer.fit_transform(data)

# END CODE HERE

all_columns = list(object_columns) + list(non_object_columns)

# Create the new DataFrame
data_scaled_labeled = pd.DataFrame(data_scaled_labeled, columns=all_columns)

**TEST** (DON'T EDIT THIS CELL)

In [15]:
# DON'T EDIT THIS CELL

data_scaled_labeled_check = pd.read_csv("/content/data_scaled_labeled_check.csv")
import numpy as np
data_scaled_labeled_values = np.sort(data_scaled_labeled.values,axis=0)
data_scaled_labeled_check_values = np.sort(data_scaled_labeled_check.values,axis=0)
if np.allclose(data_scaled_labeled_values, data_scaled_labeled_check_values, equal_nan=True):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


## Data Splitting

**CODE**

We need to drop the Weather Type column from data_scaled_labeled to get the X and pick Weather Type column to get Y

Use 90-10 split of data, i.e. 90% for training, 10% for testing

In [16]:
X = data_scaled_labeled.drop(columns = 'Weather Type')
y = data_scaled_labeled['Weather Type']


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X , y , train_size = 0.9, random_state = 10)

**TEST** (DON'T EDIT THIS CELL)

In [17]:
# DON'T EDIT THIS CELL

if (len(X_train)==11880 and len(X_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

if (len(y_train)==11880 and len(y_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed
Test Passed


In [18]:
X_train.head()

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
7017,0.0,3.0,0.0,1.315591,-1.372266,-0.120461,-1.397525,0.255445,1.813646,0.900845
12278,2.0,2.0,2.0,-0.525006,-0.084722,-0.265211,-0.207993,0.042801,-0.779410,0.159307
2859,3.0,3.0,2.0,-0.870118,1.301864,-0.337587,1.357181,-0.611801,-0.779410,-0.285616
5495,2.0,1.0,0.0,-1.732897,0.212404,-0.120461,-0.583634,3.494314,2.072951,-1.027154
8469,0.0,3.0,0.0,0.050181,-0.579931,-0.265211,-1.460132,0.455455,1.295035,0.900845


# Training and Testing
Use any suitable classifier to fit the training data and check the accuracy on test data

Don't forget to import that library

**CODE**

ACCURACY should be more than 90%,

1. Fit the model to X_train and y_train
2. Find the model score on X_test and y_test

In [19]:
# START CODE HERE
# START CODE HERE

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
# END CODE HERE

print("Accuracy:", score * 100)


Accuracy: 92.12121212121212


**TEST** (DON'T EDIT THIS CELL)

In [20]:
# DON'T EDIT THIS CELL

if (score>0.90):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


# Predicting Output (Here, Weather Type) (Optional)

In [21]:
input = X_test.head()
input

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
11923,0.0,0.0,2.0,0.050181,-2.214122,-0.916588,-1.460132,0.221304,1.813646,1.197460
2331,3.0,3.0,2.0,1.373109,-1.718913,-0.554712,-1.397525,0.462714,0.517118,0.752538
217,2.0,0.0,2.0,3.501299,1.896115,1.327043,0.511987,0.336095,-0.779410,-0.285616
1760,2.0,3.0,0.0,-0.179894,0.410487,0.458541,1.075450,-0.037580,-0.520104,-0.285616
3963,3.0,0.0,0.0,-0.179894,0.360966,0.603291,-1.147097,0.075598,-0.001493,0.307615


In [22]:
y = model.predict(input)
y

array([3., 3., 0., 1., 0.])

Here, the outputs are in numbers. We can perform inverse column transfer for getiing the string values.
